
# Galves-Löcherbach Model

"GL model assumes that the firing of the neuron is a random event, whose probability of occurrence in any time step is a firing function Φ(V) of membrane potential V. By subsuming all sources of randomness into a single function, the Galves-Löcherbach (GL) neuron model simplifies the analysis and simulation of noisy spiking neural networks." [DOI: 10.1038/srep35831]

Reproduce the results of: Antonio Galves, Eva Löcherbach, Christophe Pouzat, Errico Presutti, "A system of interacting neurons with short term synaptic facilitation". arXiv:1903.01270v3 13 Sep 2019

Tsodyks-Markram model formally equivalent for synapse (when set to no depression, only facilitation)

Morrison A., Diesmann M., Gerstner W. (2008) Phenomenological models of synaptic plasticity based on spike timing. Biological Cybernetics 98:459–478.
DOI: 10.1007/s00422-008-0233-1.


Preliminaries
-------------

In [ ]:
%matplotlib inline
from typing import Dict, Optional

import matplotlib as mpl

mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

import matplotlib.pyplot as plt

import nest
import numpy as np
import os
import random

from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

GL model with physical units
----------------------------


In [ ]:
# Neuron parameters
params = {
    'tau_m'   : 10.0,
    't_ref'   : 2.0,
    'C_m'     : 250.0,
    'V_r'     : -65.0,
    'V_reset' : -65.0,
    'a'       : 1.2,
    'b'       : 27.0,
    'V_b'     : -51.3
}

In [ ]:
nestml_gl_exp_model = """
neuron gl_exp:
    state:
        refr_spikes_buffer mV = 0 mV
        refr_tick integer = 0    # Counts number of tick during the refractory period
        V_m mV = V_r     # Membrane potential

    equations:
        kernel G = delta(t)
        V_m' = -(V_m - V_r) / tau_m + (mV / ms) * convolve(G, spikes) + (I_e + I_stim) / C_m

    parameters:
        tau_m   ms   = 10 ms              # Membrane time constant
        C_m     pF   = 250 pF             # Capacity of the membrane
        t_ref   ms   = 2 ms               # Duration of refractory period
        tau_syn ms   = 0.5 ms             # Time constant of synaptic current
        V_r     mV   = -65 mV             # Resting membrane potential
        V_reset mV   = -65 mV             # Reset potential of the membrane
        b     real   = 27                 # Parameter for the exponential curve
        a       mV   = 5 mV               # Parameter for the exponential curve
        V_b mV       = -51.3 mV             # Membrane potential at which phi(V)=1/b
        with_refr_input boolean = false # If true, do not discard input during refractory period.
        reset_after_spike boolean = true

        # constant external input current
        I_e pA = 0 pA

    internals:
        RefractoryCounts integer = steps(t_ref) # refractory time in steps

    input:
        spikes <- spike
        I_stim pA <- continuous

    output:
        spike

    function phi(V_m mV) real:
        return ((1/b) * exp((V_m - V_b)/mV/a))

    update:
        if refr_tick == 0: # neuron not refractory
            integrate_odes()

            # if we have accumulated spikes from refractory period,
            # add and reset accumulator
            if with_refr_input and refr_spikes_buffer != 0.0 mV:
                V_m += refr_spikes_buffer
                refr_spikes_buffer = 0.0 mV

        else: # neuron is absolute refractory
            # read spikes from buffer and accumulate them, discounting
            # for decay until end of refractory period
            # the buffer is clear automatically
            if with_refr_input:
                refr_spikes_buffer += spikes * exp(-refr_tick * h / tau_m) * mV * s
            refr_tick -= 1

        tmp real = phi(V_m)
        println("{V_m} ---> {tmp}")
            
        if random_uniform(0, 1) <= 1E-3 * resolution() * phi(V_m):
            refr_tick = RefractoryCounts
            if reset_after_spike:
                V_m = V_reset
            emit_spike()
"""

In [ ]:
module_name, neuron_model_name = NESTCodeGeneratorUtils.generate_code_for(
    nestml_neuron_model=nestml_gl_exp_model,
    logging_level="INFO"  # try "INFO" for more debug information
)

nest.Install(module_name)

In [ ]:
def measure_numerical_Phi_function(neuron_model_name, U_min=0., U_max=10., neuron_model_params=None, neuron_membrane_potential_name="U"):
    nest.ResetKernel()
    nest.resolution = 1.   # check that results are independent of resolution...
    
    t_stop = 25000.
    
    U_range = np.linspace(U_min, U_max, 12)
    n_spikes = np.nan * np.ones_like(U_range)
    for i, U in enumerate(U_range):
        neuron = nest.Create(neuron_model_name)
        if neuron_model_params:
            neuron.set(neuron_model_params)
            
        neuron.set({neuron_membrane_potential_name: U})
        assert neuron.get(neuron_membrane_potential_name) == U
    
        sr = nest.Create('spike_recorder')
        nest.Connect(neuron, sr)
    
        nest.Simulate(t_stop)
        assert neuron.get(neuron_membrane_potential_name) == U
    
        n_spikes[i] = len(sr.events["times"])
    
    spike_rate = n_spikes / (t_stop / 1E3)

    return U_range, spike_rate

In [ ]:

# theoretical Phi vs U
U_range_theory = np.linspace(-60., -45., 100)
Phi_of_U_theory = (1 / params['b']) * np.exp((U_range_theory - params['V_b']) / params['a'])

# numerical Phi vs U
U_range_numeric, spike_rate_numeric = measure_numerical_Phi_function(neuron_model_name=neuron_model_name,
                                                                     U_min=-60.,
                                                                     U_max=-45.,
                                                                     neuron_model_params={"reset_after_spike": False,
                                                                                          "a": params['a'],
                                                                                          "b": params['b'],
                                                                                          "V_b": params['V_b'],
                                                                                          "tau_m": 1E99},
                                                                     neuron_membrane_potential_name="V_m")

In [ ]:
fig, ax = plt.subplots()
ax.plot(U_range_theory, Phi_of_U_theory, label="theory")
ax.plot(U_range_numeric, spike_rate_numeric, marker="o", label="numeric")
ax.legend()
ax.set_xlabel("$U$")
ax.set_ylabel("Firing rate [Hz]")

In [ ]:
spike_rate_numeric

Normalized model with calcium dynamics
---------------------------------------------------

The `R` variable represents the calcium concentration of the presynaptic neuron. For convenience (and at the cost of some redundancy), we store it here in the NEST synapse object.

In [ ]:
nestml_gl_ca_synapse_model = '''
synapse syn_gl_ca:
  state:
    R_pre real = 0.

  parameters:
    the_delay ms = 1 ms  @nest::delay   # !!! cannot have a variable called "delay"
    lmbda real = 2.1555489309487914   # residual calcium decay rate

  onReceive(incoming_spikes):
    R_pre += 1
    deliver_spike(R_pre - 1, the_delay)

  input:
    incoming_spikes <- spike
  
  output:
    spike
  
  update:
    R_pre *= exp(-lmbda * 1E-3 * resolution())   # leakage
'''

Neuron:

In [ ]:
nestml_gl_ca_neuron_model = '''
neuron gl_ca:
  state:
    U real = 0     # membrane potential

  parameters:
    a real = 3.
    alpha_over_N real = 1.0777744654743957   # synaptic strength
    beta real = 50     # membrane potential leak
    reset_after_spike boolean = true

  input:
    incoming_spikes <- spike

  output:
    spike

  function phi(U real) real:
    if U <= 0:
      return 0

    #tmp real = (4 * a) / (1 + exp(a - U)) - (4 * a) / (1 + exp(a))
    #println("phi({U}) = {tmp}")
    return (4 * a) / (1 + exp(a - U)) - (4 * a) / (1 + exp(a))

  equations:
    kernel K = delta(t)
    
    # R is presynaptic neuron's R and is passed as the spike weight by the synapse
    U' = -U / (1E3 / beta) + alpha_over_N * convolve(K, incoming_spikes) / ms

  update:
    # integrate spike input
    integrate_odes()

    # emit spike?
    if random_uniform(0, 1) <= 1E-3 * resolution() * phi(U):
      emit_spike()
      if reset_after_spike:
          U = 0   # reset membrane potential
'''

In [ ]:
module_name, neuron_model_name, synapse_model_name = NESTCodeGeneratorUtils.generate_code_for(
    nestml_neuron_model=nestml_gl_ca_neuron_model,
    nestml_synapse_model=nestml_gl_ca_synapse_model,
    logging_level="INFO"  # try "INFO" for more debug information
)

nest.Install(module_name)

### Postsynaptic response

Note that due to the residual calcium dynamics, the synapse is facilitating.

The decay should correspond to the $\beta$ parameter.

When a neuron spikes, it increases the potential of each postsynaptic partner by $\alpha R/N$.

In [ ]:
def measure_postsynaptic_response(neuron_model: str,
                                  synapse_model: str,
                                  t_stop: float = 2250.,
                                  V_m_specifier: str = "V_m",
                                  custom_model_opts: Optional[Dict] = None):
    spike_times = np.array([100., 200., 250., 2000.])

    nest.ResetKernel()
    neuron = nest.Create(neuron_model, params=custom_model_opts)
    neuron.alpha_over_N = 1E-6   # a very low value, to prevent the neuron from spiking
    #dc = nest.Create("dc_generator", params={"amplitude": 1E12 * I_stim})  # 1E12: convert A to pA
    #nest.Connect(dc, neuron)
    spike_generator = nest.Create("spike_generator", params={"spike_times": spike_times})
    nest.Connect(spike_generator, neuron, syn_spec={'synapse_model': synapse_model})

    multimeter = nest.Create('multimeter')
    nest.SetStatus(multimeter, {"record_from": [V_m_specifier]})
    nest.Connect(multimeter, neuron)

    sr = nest.Create('spike_recorder')
    nest.Connect(neuron, sr)

    nest.Simulate(t_stop)

    ts = multimeter.events["times"]
    Vms = multimeter.events[V_m_specifier]
    
    return ts, Vms

In [ ]:
ts, Vms_gl = measure_postsynaptic_response("gl_ca_nestml", "syn_gl_ca_nestml", V_m_specifier="U")

fig, ax = plt.subplots()
ax.set_xlabel("$t$ [ms]")
ax.plot(ts, Vms_gl, label="gl")
ax.set_ylabel("U")
ax.legend()

### Firing rate

This should correspond to the Phi(U) function in the neuron (see plot below for the theoretical curve).

In [ ]:
#theoretical Phi vs U
a = 3
U_range_theory = np.linspace(0., 10., 100)
Phi_of_U_theory = (4 * a) / (1 + np.exp(a - U_range_theory)) - (4 * a) / (1 + np.exp(a))

# numerical Phi vs U
U_range_numeric, spike_rate_numeric = measure_numerical_Phi_function(neuron_model_name=neuron_model_name,
                                                                     neuron_model_params={"reset_after_spike": False,
                                                                                          "beta": 1E-99,   # a very low value, to prevent the membrane potential from decaying
                                                                                          "a": 3.})

In [ ]:
fig, ax = plt.subplots()
ax.plot(U_range_theory, Phi_of_U_theory, label="theory")
ax.plot(U_range_numeric, spike_rate_numeric, marker="o", label="numeric")
ax.legend()
ax.set_xlabel("$U$")
ax.set_ylabel("Firing rate [Hz]")

### Network dynamics

In [ ]:
foo=None
def run_simulation_in_chunks(sim_chunks, sim_time, syn_recordables, neurons):
    global foo
    sim_time_per_chunk = sim_time / sim_chunks

    # Init log to collect the values of all recordables
    log = {}
    log["t"] = []

    # Initialize all the arrays
    # Additional one entry is to store the trace value before the simulation begins
    for rec in syn_recordables:
        log[rec] = (sim_chunks + 1) * [[]]

    # Get the value of trace values before the simulation
    syn = nest.GetConnections(target=neurons, synapse_model="syn_gl_ca_nestml")
    print(str(len(syn)) + " synapses in the network")
    foo=syn
    for rec in syn_recordables:
        log[rec][0] = syn.get(rec)
        
    log["t"].append(nest.GetKernelStatus("biological_time"))

    # Run the simulation in chunks
    for i in range(sim_chunks):
        sim_start_time = i * sim_time_per_chunk
        sim_end_time = sim_start_time + sim_time_per_chunk

        nest.Simulate(np.round(sim_time/sim_chunks))
        
        # log current values
        log["t"].append(nest.GetKernelStatus("biological_time"))

        # Get the value of trace after the simulation
        for rec in syn_recordables:
            log[rec][i + 1] = syn.get(rec).copy()
            
    #nest.Cleanup()
    
    return log


In [ ]:
N = 100      # Number of neurons in network XXX: 1000 in the original paper
conn_prob = 1.

a = 2.

beta = 50.
lmbda = 10.
alpha = beta * lmbda

U_0 = 1.#0.79555   # initial membrane potential
R_0 = .2#1.   # initial residual calcium

U_range = .1 * U_0
R_range = .1 * R_0

In [ ]:
nest.ResetKernel()
nest.resolution = .1 # [ms]

sim_time = 500.   # [ms] XXX: 120 second original paper
chunk_length = 10.   # [ms]
n_chunks = int(sim_time / chunk_length)
syn_recordables = ["R_pre"]

pop = nest.Create("gl_ca_nestml", N)
pop.a = a
pop.alpha_over_N = alpha / N
pop.beta = beta
pop.U = [U_0 + U_range * (random.random() - .5) for _ in range(N)]

conn_spec_dict = {'rule': 'pairwise_bernoulli', 'p': conn_prob, 'allow_autapses': False}
nest.Connect(pop, pop, conn_spec_dict, syn_spec={'synapse_model': 'syn_gl_ca_nestml', 'lmbda': lmbda})

#syn_recordables = []
#nest.Connect(pop, pop, conn_spec_dict, syn_spec={'synapse_model': 'static_synapse'})


syn = nest.GetConnections(target=pop, synapse_model="syn_gl_ca_nestml")
N_syn = len(syn)
syn.R_pre = [R_0 + R_range * (random.random() - .5) for _ in range(N_syn)]

multimeter = nest.Create("multimeter")
multimeter.set({"record_from": ["U"], "interval": nest.resolution})
nest.Connect(multimeter, pop)

sr = nest.Create('spike_recorder')
nest.Connect(pop, sr)

log = run_simulation_in_chunks(n_chunks, sim_time, syn_recordables, pop)


"""nest.Simulate(sim_time)

neuron_ids = np.unique(multimeter.get("events")["senders"])

fig, ax = plt.subplots(nrows=2)

U_avg = np.zeros(len(np.unique( multimeter.get("events")["times"])))
# R_avg = np.zeros_like(U_avg)
for neuron_id in range(1, N+1):
    idx = np.where(neuron_id == multimeter.get("events")["senders"])
    times = multimeter.get("events")["times"][idx]
    U = multimeter.get("events")["U"][idx]
    #R = multimeter.get("events")["R"][idx]
    
    U_avg += U / N
    #R_avg += R / N

    if neuron_id < 100:
        ax[0].plot(U, label="U")
#         ax[1].plot(R, label="R")

ax[0].plot(U_avg, linewidth=4, linestyle="--", c="black")
ax[0].set_ylabel("U")
# ax[1].plot(R_avg, linewidth=4, linestyle="--", c="black")
ax[1].set_ylabel("R")

for _ax in ax:
    _ax.set_xlim(0, sim_time)
ax[-1].set_xlabel("Time [ms]")"""

In [ ]:
times = log["t"]
R_pre = np.array(log["R_pre"])
R_pre_avg = np.mean(R_pre, axis=1)


fig, ax = plt.subplots()
ax.plot(times, R_pre[:, ::N], alpha=.5)   # XXX: plot only every N-th line
ax.set_xlim(0, sim_time)
ax.set_xlabel("Time [ms]")
ax.plot(times, R_pre_avg, linewidth=4, linestyle="--", c="black", label="mean")
ax.legend()
fig.suptitle("R")

In [ ]:
fig, ax = plt.subplots()
ax.scatter(sr.events["times"], sr.events["senders"])
ax.set_ylim(0, N)
ax.set_xlim(0, sim_time)
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Neuron")

In [ ]:
avg_firing_rate = len(sr.events["times"]) / N / (sim_time / 1E3)
print("Network average firing rate: " + str(avg_firing_rate) + " Hz")

In [ ]:
neuron_ids = np.unique(multimeter.get("events")["senders"])

fig, ax = plt.subplots()

U_avg = np.zeros(len(np.unique(multimeter.get("events")["times"])))
for neuron_id in range(1, N+1):
    idx = np.where(neuron_id == multimeter.get("events")["senders"])
    times = multimeter.get("events")["times"][idx]
    U = multimeter.get("events")["U"][idx]
    
    U_avg += U / N

    if neuron_id < 100:
        ax.plot(times, U, label="U")

ax.plot(times, U_avg, linewidth=4, linestyle="--", c="black")
ax.set_ylabel("U")
ax.set_xlabel("Time [ms]")
fig.suptitle("U")

### Comparison to theory

In [ ]:
def phi(x):
    return (4 * a) / (1 + np.exp(a - x)) - (4 * a) / (1 + np.exp(a))

def nullcline1(x):
    return phi(x)/lmbda

def nullcline2(x):
    return beta/alpha*x/phi(x)

U_vec = np.logspace(np.log10(.1), np.log10(150), 100)

nullcline1_vec = nullcline1(U_vec)
nullcline2_vec = nullcline2(U_vec)

fig, ax = plt.subplots()
ax.loglog(U_vec, nullcline1_vec)
ax.loglog(U_vec, nullcline2_vec)

U_avg_intrp = np.interp(np.linspace(0, len(U_avg), 100),
                        np.arange(len(U_avg)),
                        U_avg)

R_pre_intrp = np.interp(np.linspace(0, len(R_pre_avg), 100),
                        np.arange(len(R_pre_avg)),
                        R_pre_avg)

R_pre_intrp[ R_pre_intrp <= 1] = 1.000001

#ax.loglog(U_avg[::200], R_pre_avg[:50] - 1)
ax.loglog(U_avg_intrp, R_pre_intrp - 1)
ax.loglog(U_avg[-1], R_pre_avg[-1] - 1, marker="o")

#ax.scatter(U_avg[:199], R_pre_avg[:199])

ax.set_xlabel('Membrane potential')
ax.set_ylabel('Residual calcium')
ax.set_xlim(np.amin(U_vec), np.amax(U_vec))
ax.set_ylim(.01, 6)

#ax.plot(U_avg, R_avg)

In [ ]:
R_pre_intrp

## Acknowledgements

...


## References

...


## Copyright

This file is part of NEST.

Copyright (C) 2004 The NEST Initiative

NEST is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 2 of the License, or (at your option) any later version.

NEST is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with NEST.  If not, see <http://www.gnu.org/licenses/>.
